# What's New in C# 11

[.NET Conf Thailand 2022](https://dotnetconfth.com/)

Will Fuqua - [Jetabroad (Thailand)](https://bkkthailand.jetabroad.com/)


### Raw String Literals

In [1]:
// normal string -- hard to read!
var normalString = "{\n    \"hello\": \"world\"\n}";
Console.WriteLine(normalString);

// verbatim string -- better, but still not perfect because of duplicated ""
Console.WriteLine(@"{
    ""hello"": ""world""
}");

// TODO: verbatim interpolated string, not new, but a bit confusing.
var greeting = "hello";
Console.WriteLine(@"{
    ""hello"": ""world""
}");

// TODO: raw string literal
Console.WriteLine("""
{
    "hello": "world"
}
""");

// TODO: try interpolation with line breaks


{
    "hello": "world"
}
{
    "hello": "world"
}
{
    "hello": "world"
}
{
    "hello": "world"
}


### Generic Attributes

In [29]:
using System.IO;

// our serializer attribute takes the type of a serializer
// TODO: change to generic attribute and take advantage of 
[CustomSerializer(typeof(DogSerializer))]
public class Dog
{
}

public class CustomSerializerAttribute : Attribute
{
    public Type Serializer { get; set; }

    public CustomSerializerAttribute(Type serializer) => 
        Serializer = serializer;
}

public class DogSerializer : ISerializer<Dog>
{
    public Stream Serialize(Dog obj) => new MemoryStream();
}

public interface ISerializer<T>
{
    Stream Serialize(T obj);
}

### Static Abstract Members

- Replace both reflection and code generation with compile-time code.
- Allow more general generic implementations

In [34]:
// The problem: How do you store "metadata" on a type?

// via a static property? But that doesn't work with generics
public class User
{
    public static string Table => "dbo.User";
}

// via an attribute? but that requires reflection :(
[Table("dbo.Users")]
public class Users
{
}

public void DeleteRecords<T>()
{
    var tableName = ((TableAttribute)Attribute.GetCustomAttribute(typeof(T), typeof(TableAttribute))).Table;
    var sqlString = $"DELETE FROM {tableName}";
}

public class TableAttribute : Attribute
{
    public string Table { get; set; }
    public TableAttribute(string table) =>
        this.Table = table;
}

In [ ]:
public class User : IDatabaseEntity<User>
{
    public static string Table => "dbo.Users";
}

// TODO: add static abstract member
public interface IDatabaseEntity<T>
{
}

public void DeleteRecords<T>() where T : IDatabaseEntity<T>
{
}

#### Static Abstract Members - Generic Math

- This is an application of the above feature

In [46]:
int SumInt(params int[] numbers)
{
    var sum = 0;
    foreach(var num in numbers)
    {
        sum += num;
    }
    return sum;
}

return SumInt(1, 2, 3);

6

In [47]:
// https://github.com/dotnet/runtime/blob/main/src/libraries/System.Private.CoreLib/src/System/Int32.cs

using System.Numerics;

T Sum<T>(params T[] numbers)
    where T : INumber<T>
{
    T sum = T.AdditiveIdentity;
    foreach(var num in numbers)
    {
        sum += num;
    }
    return sum;
}

return Sum(1, 2, 3);

6

### List Pattern Matching

- Pattern matching was introduced in C# 7
- Every version of C# has added additional patterns.
- C# 11 adds... lists!

In [69]:
var name = "Peter Parker";

public string FormatName(string name) => name.Split() switch
{
    [] => "anonymous",
    [var first] => first,
    [var first, var surname] => $"{surname}, {first}",
    [var first, ..var middleNames, var surname] => $"{first} {string.Join(" ", middleNames)}, {surname}",
};

return FormatName(name);

Parker, Peter

In [68]:

if(new[] { 1, 2, 3 } is [>0, >1, >2]) {
    return true;
}

True

### Required Members

- Improves object initializers
- Reduces the need for long lists of constructors

In [ ]:
//#nullable enable

public class Person
{
    public string FirstName { get; set; }
    public string MiddleName { get; set; }
    public string LastName { get; set; }

    /*
    public Person(string firstName, string middleName, string lastName)
    {
        FirstName = firstName;
        MiddleName = middleName;
        LastName = lastName;
    }
    */
}

public class DotNetDeveloper : Person
{
    public bool UsesDarkMode { get; set; } = true;
}

/*
var p = new Person
{
    FirstName = "A",
};
*/


### UTF-8 String Literals

In [3]:
// utf-16 by default
Print(System.Text.Encoding.Unicode.GetBytes("ABC"));

// old style
Print(System.Text.Encoding.UTF8.GetBytes("ABC"));

// new style
Print("ABC"u8.ToArray());

void Print(byte[] bytes) => Console.WriteLine(string.Join(", ", bytes));


65, 0, 66, 0, 67, 0
65, 66, 67
65, 66, 67


### nameof - extended scope

In [4]:
#nullable enable
using System.Diagnostics.CodeAnalysis;

// this has a nullability warning
string screaming = UpperCase("hello");

// we can fix the warning by adding the below declaration. nameof() didn't work in this case in previous versions of C#
//[return: NotNullIfNotNull(nameof(input))]
string? UpperCase(string? input) => input?.ToUpper();

### Auto default struct

In [ ]:
public readonly struct Measurement
{
    public double Value { get; init; }
    public string Description { get; init; }

    public Measurement(double value, string description)
    {
        Value = value;
        Description = description;
    }

    // Before in C#, this would not compile, we'd have the error:
    //     'Measurement.Value' must be fully assigned before control is returned to the caller.
    public Measurement(string description)
    {
        Description = description;
    }
}

### File local types (access modifier)

In [ ]:
// to visual studio!

### Pattern Match Spans

In [15]:
// to visual studio (again)!

### Improved performance of method group conversion to delegate

In [6]:
int[] numbers = { -2, -1, 0, 1, 2 };

bool IsEven(int n) => n % 2 == 0;

// in previous C# versions, these have the same behavior, but one of them is slower!

// in C# 11, the following two have the same behavior and performance.

var even1 = numbers.Where(n => IsEven(n)).ToList();

var even2 = numbers.Where(IsEven).ToList();


### Numeric int pointer alias

In [14]:
using System.Runtime.InteropServices;

var myStr = "I seem to be turned around!";
var handle = GCHandle.Alloc(myStr);
nint parameter = (IntPtr)handle; // nint is new alias for IntPtr (as is nuint for UIntPtr)
Console.WriteLine(parameter);
handle.Free();

1628828078104


### Low-level Struct Improvements


A huge list of improvements, [documented here](https://github.com/dotnet/csharplang/blob/main/proposals/csharp-11.0/low-level-struct-improvements.md).

As a summary, you can now implement your own custom types that are as fast as `Span<T>`.

Previously, `Span<T>` relied on internal runtime behavior, but it's been generalized and any struct can take part.